In [35]:
import pandas as pd
import numpy as np

class ModelContributionAmounts:
    def __init__(self, minimum_eligible_amount, apy):
        """
        Initialize the contribution amounts model.

        Parameters:
        - minimum_eligible_amount (float): Minimum treatment cost to be eligible for the program.
        - apy (float): Annual Percentage Yield for interest calculations.
        """
        self.minimum_eligible_amount = minimum_eligible_amount
        self.apy = apy

    def process_transactions(self, transaction_df, conversion_rate, monthly_payment_rate):
        """
        Process transactions to determine contribution program eligibility and generate schedules.

        Parameters:
        - transaction_df (pd.DataFrame): Input transaction DataFrame with 'Customer ID', 'Period', and 'Revenue'.
        - conversion_rate (float): Probability of a transaction being keen for conversion.
        - monthly_payment_rate (float): Percentage of the treatment price as the monthly contribution.

        Returns:
        - pd.DataFrame: Updated transaction DataFrame with a 'Convert' column.
        - pd.DataFrame: Resulting contribution schedule DataFrame.
        """
        
        transaction_df['Period'] = pd.to_datetime(transaction_df['Period'])
        
        # Initialize columns
        transaction_df['Convert'] = False

        # Determine the earliest possible date
        min_period = transaction_df['Period'].min()

        # Initialize result DataFrame
        result_data = []

        for idx, row in transaction_df.iterrows():
            customer_id = row['Customer ID']
            treatment_date = row['Period']
            treatment_price = row['Revenue']

            # Skip if treatment price is below minimum eligible amount
            if treatment_price < self.minimum_eligible_amount:
                continue

            # Apply conversion rate
            if np.random.rand() > conversion_rate:
                continue

            # Calculate monthly payment
            monthly_payment = treatment_price * monthly_payment_rate
            nper = 0
            accrued_interest = 0
            total_future_value = 0

            # Calculate nper iteratively to include accrued interest
            while total_future_value < treatment_price:
                nper += 1
                future_value = monthly_payment * (1 + self.apy / 12) ** (nper - 1)
                total_future_value += future_value
                accrued_interest += future_value - monthly_payment

            # Check if nper satisfies the time delta condition
            time_delta = (treatment_date - min_period).days
            if time_delta < 30 * nper:
                continue

            # Eligible transaction, mark as Convert
            transaction_df.at[idx, 'Convert'] = True

            # Generate contribution schedule
            first_deposit_date = treatment_date - pd.Timedelta(days=30 * (nper - 1))
            for i in range(nper - 1):
                deposit_date = first_deposit_date + pd.Timedelta(days=30 * i)
                result_data.append({
                    'Period': deposit_date,
                    'Customer ID': customer_id,
                    'Amount': monthly_payment,
                    'Type': 'Deposit'
                })

            # Add final payment row
            remaining_fee = treatment_price - total_future_value + accrued_interest
            result_data.append({
                'Period': treatment_date,
                'Customer ID': customer_id,
                'Amount': remaining_fee,
                'Type': 'Final Payment'
            })

        # Create result DataFrame
        result_df = pd.DataFrame(result_data)
        return transaction_df, result_df

    def validate_result(self, updated_transaction_df, result_df):
        """
        Validate that the result DataFrame correctly calculates deposits and interest.

        Parameters:
        - updated_transaction_df (pd.DataFrame): Transaction DataFrame with 'Convert' column.
        - result_df (pd.DataFrame): Contribution schedule DataFrame.

        Returns:
        - list: Validation issues, if any.
        """
        validation_issues = []

        for customer_id in updated_transaction_df.loc[updated_transaction_df['Convert'] == True, 'Customer ID']:
            # Filter result_df for this customer
            customer_payments = result_df[result_df['Customer ID'] == customer_id]

            # Separate deposit and final payment rows
            deposits = customer_payments[customer_payments['Type'] == 'Deposit']
            final_payment_row = customer_payments[customer_payments['Type'] == 'Final Payment']

            # Recalculate accrued interest
            treatment_date = final_payment_row['Period'].iloc[0]
            total_accrued_interest = 0
            for _, deposit_row in deposits.iterrows():
                deposit_date = deposit_row['Period']
                months_remaining = ((treatment_date - deposit_date).days) // 30
                future_value = deposit_row['Amount'] * (1 + self.apy / 12) ** months_remaining
                accrued_interest = future_value - deposit_row['Amount']
                total_accrued_interest += accrued_interest

            # Calculate totals
            total_deposits = deposits['Amount'].sum()
            final_payment = final_payment_row['Amount'].iloc[0]
            treatment_price = updated_transaction_df.loc[
                (updated_transaction_df['Customer ID'] == customer_id) & (updated_transaction_df['Convert'] == True), 'Revenue'
            ].iloc[0]

            # Compare with treatment price
            calculated_total = total_deposits + total_accrued_interest + final_payment
            if not np.isclose(calculated_total, treatment_price, atol=0.01):
                validation_issues.append({
                    'Customer ID': customer_id,
                    'Total Deposits': total_deposits,
                    'Accrued Interest': total_accrued_interest,
                    'Final Payment': final_payment,
                    'Treatment Price': treatment_price,
                    'Calculated Total': calculated_total
                })

        return validation_issues

In [36]:


transactions_df = pd.read_csv('forecast_df_treatment.csv')

# Initialize the model
contribution_model = ModelContributionAmounts(minimum_eligible_amount=1000, apy=0.05)

# Process transactions
updated_transactions_df, result_df = contribution_model.process_transactions(
    transactions_df,
    conversion_rate=0.5,
    monthly_payment_rate=0.2
)



In [37]:
result_df

,Period,Customer ID,Amount,Type
0,2025-02-08,Patient 587,2.084000e+02,Deposit
1,2025-03-10,Patient 587,2.084000e+02,Deposit
2,2025-04-09,Patient 587,2.084000e+02,Deposit
3,2025-05-09,Patient 587,2.084000e+02,Deposit
4,2025-06-08,Patient 587,5.684342e-14,Final Payment
...,...,...,...,...
90,2025-08-11,Patient 351,5.513200e+03,Deposit
91,2025-09-10,Patient 351,5.513200e+03,Deposit
92,2025-10-10,Patient 351,5.513200e+03,Deposit
93,2025-11-09,Patient 351,5.513200e+03,Deposit


In [38]:
result_df[result_df['Customer ID'] == 'Patient 587']

,Period,Customer ID,Amount,Type
0,2025-02-08,Patient 587,2.084000e+02,Deposit
1,2025-03-10,Patient 587,2.084000e+02,Deposit
2,2025-04-09,Patient 587,2.084000e+02,Deposit
3,2025-05-09,Patient 587,2.084000e+02,Deposit
4,2025-06-08,Patient 587,5.684342e-14,Final Payment


In [39]:
updated_transactions_df[updated_transactions_df['Customer ID'] == 'Patient 587']

,Period,Treatment,Revenue,Expense,Customer ID,Convert
703,2025-06-08,664,1042.0,128.380000,Patient 587,True
1249,2025-10-05,171,58.0,5.876667,Patient 587,False


In [40]:
def validate_result(updated_transaction_df, result_df, apy):
    validation_issues = []

    # Group payments by Customer ID
    for customer_id in updated_transaction_df.loc[updated_transaction_df['Convert'] == True, 'Customer ID']:
        # Filter result_df for this customer
        customer_payments = result_df[result_df['Customer ID'] == customer_id]

        # Separate deposit and final payment rows
        deposits = customer_payments[customer_payments['Type'] == 'Deposit']
        final_payment_row = customer_payments[customer_payments['Type'] == 'Final Payment']

        # Recalculate accrued interest
        treatment_date = final_payment_row['Period'].iloc[0]
        total_accrued_interest = 0
        for _, deposit_row in deposits.iterrows():
            deposit_date = deposit_row['Period']
            months_remaining = ((treatment_date - deposit_date).days) // 30
            future_value = deposit_row['Amount'] * (1 + apy / 12) ** months_remaining
            accrued_interest = future_value - deposit_row['Amount']
            total_accrued_interest += accrued_interest

        # Calculate totals
        total_deposits = deposits['Amount'].sum()
        final_payment = final_payment_row['Amount'].iloc[0]
        treatment_price = updated_transaction_df.loc[
            (updated_transaction_df['Customer ID'] == customer_id) & (updated_transaction_df['Convert'] == True), 'Revenue'
        ].iloc[0]

        # Compare with treatment price
        calculated_total = total_deposits + total_accrued_interest + final_payment
        if not np.isclose(calculated_total, treatment_price, atol=0.01):
            validation_issues.append({
                'Customer ID': customer_id,
                'Total Deposits': total_deposits,
                'Accrued Interest': total_accrued_interest,
                'Final Payment': final_payment,
                'Treatment Price': treatment_price,
                'Calculated Total': calculated_total
            })

    return validation_issues


In [41]:
validate_result(updated_transactions_df, result_df, apy=0.05)

[{'Customer ID': 'Patient 587',
  'Total Deposits': 833.6,
  'Accrued Interest': 8.719589327859751,
  'Final Payment': 5.684341886080802e-14,
  'Treatment Price': 1042.0,
  'Calculated Total': 842.3195893278598},
 {'Customer ID': 'Patient 462',
  'Total Deposits': 1062.4,
  'Accrued Interest': 11.1128739226466,
  'Final Payment': -3.410605131648481e-13,
  'Treatment Price': 1328.0,
  'Calculated Total': 1073.5128739226461},
 {'Customer ID': 'Patient 239',
  'Total Deposits': 13604.800000000001,
  'Accrued Interest': 142.3083839823248,
  'Final Payment': -1.8189894035458565e-12,
  'Treatment Price': 17006.0,
  'Calculated Total': 13747.108383982324},
 {'Customer ID': 'Patient 727',
  'Total Deposits': 1331.2,
  'Accrued Interest': 13.924564915123426,
  'Final Payment': -5.684341886080802e-14,
  'Treatment Price': 1664.0,
  'Calculated Total': 1345.1245649151235},
 {'Customer ID': 'Patient 33',
  'Total Deposits': 1179.2,
  'Accrued Interest': 12.334620603901328,
  'Final Payment': -5.68